# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
print(f"module_path: {module_path}")
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

module_path: /home/beegass/Documents/Coding/s4mer


In [2]:
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "False"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"

In [3]:
## import packages
import jax
import jax.numpy as jnp
from src.models.hippo.gu_hippo import HRHiPPO_LSI, HRHiPPO_LTI
from src.models.hippo.hippo import HiPPOLSI, HiPPOLTI
from src.data.process import whitesignal
import einops
import numpy as np
import torch

In [4]:
print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [5]:
print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [6]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [7]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [8]:
num_copies = 10
subkeys = jax.random.split(key, num=num_copies)
key = subkeys[0]

## Test HiPPO Operators

In [9]:
def test_hippo_operator(
    hippo, gu_hippo, random_input, key, s_or_t="lti", print_all=False
):
    x_tensor = torch.tensor(random_input, dtype=torch.float32)
    x_jnp = jnp.asarray(x_tensor, dtype=jnp.float32)  # convert torch array to jax array

    # My Implementation
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"----------------------------My {s_or_t} Implementation Outputs----------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_jnp = einops.rearrange(x_jnp, "batch seq_len -> batch seq_len 1")

    params = hippo.init(key, f=x_jnp)
    c = hippo.apply(params, f=x_jnp)

    # Gu's HiPPO LegS
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Gu's {s_or_t} Implementation Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_tensor = torch.moveaxis(x_tensor, 0, 1)
    GU_cs, GU_c_k = gu_hippo(x_tensor, fast=False)

    gu_c = None
    if s_or_t == "lti":
        gu_c = einops.rearrange(GU_c_k, "batch N -> batch 1 N")
    elif s_or_t == "lsi":
        gu_c = einops.rearrange(GU_cs, "seq_len batch N -> batch seq_len 1 N")
    gu_c = jnp.asarray(gu_c, dtype=jnp.float32)  # convert torch array to jax array

    if print_all:
        print(
            f"------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Testing {s_or_t} Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------"
        )

    co_flag = True
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):

            co_check = None
            if s_or_t == "lsi":
                co_check = jnp.allclose(
                    c[i, j, :, :], gu_c[i, j, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"c[{i},{j},:,:]:\n{c[i,j,:,:]}")
                    print(f"gu_c[{i},{j},:,:]:\n{gu_c[i,j,:,:]}")
                    print(f"coefficients check: {co_check}\n")

            elif s_or_t == "lti":
                co_check = jnp.allclose(
                    c[i, j, :], gu_c[i, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"c[{i},{j},:]:\n{c[i,:,:]}")
                    print(f"gu_c[{i},{j},:]:\n{gu_c[i,j,:]}")
                    print(f"coefficients check: {co_check}\n")

            if co_check == False:
                co_flag = False

    if not print_all:
        print(f"\n------------------------------------------------------------")
        print(f"---------- The Coefficients Test Passed: {co_flag} ----------")
        print(f"-------------------------------------------------------------\n\n")

    if print_all:
        print(f"c:\n {c}")
        print(f"c shape:\n {c.shape}")

        print(f"gu_c:\n {gu_c}")
        print(f"gu_c shape:\n {gu_c.shape}")

In [10]:
def test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    # N = 256
    # L = 128

    T = 3
    freq = 3
    step = 1e-3
    L = int(T / step)

    batch_size = 1
    data_size = L
    input_size = 1

    N = 64

    u = whitesignal(T, step, freq, batch_shape=(batch_size,))

    x_np = np.asarray(u)

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = HRHiPPO_LTI(
        N=N,
        method=the_measure,
        dt=step,
        T=T,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = HRHiPPO_LSI(
            N=N,
            method="legs",
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate My HiPPOs -----------------------------
    # ----------------------------------------------------------------------------------
    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPOLTI(
        N=N,
        step_size=step,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        basis_size=T,
        dtype=jnp.float32,
        unroll=False,
    )  # Bryan's

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPOLSI(
            N=N,
            max_length=L,
            step_size=step,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            dtype=jnp.float32,
            unroll=True,
        )  # Bryan's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Test HiPPO Operators ------------------------------
    # ----------------------------------------------------------------------------------

    print(f"Testing Coeffiecients for {alpha} LTI HiPPO-{the_measure}")

    test_hippo_operator(
        hippo=hippo_lti,
        gu_hippo=gu_hippo_lti,
        random_input=x_np,
        key=subkeys[5],
        s_or_t="lti",
        print_all=print_all,
    )

    if the_measure == "legs":
        print(f"\n\nTesting Coeffiecients for {alpha} LSI HiPPO-{the_measure}")

        test_hippo_operator(
            hippo=hippo_lsi,
            gu_hippo=gu_hippo_lsi,
            random_input=x_np,
            key=subkeys[6],
            s_or_t="lsi",
            print_all=print_all,
        )

    print(f"end of test for HiPPO-{the_measure} model")

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [11]:
print_all = False

### Testing (LTI and LSI) Operators With Forward Euler Transform

#### LegS

In [12]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------




Testing Coeffiecients for 0.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------


end of test for HiPPO-legs model


#### LegT

In [13]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.0 transform
Creating HiPPO-legt LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-legt


/home/beegass/.cache/pypoetry/virtualenvs/s4mer-pkg-jZnBSgjq-py3.8/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:5117: RuntimeWarning: overflow encountered in cast
  return binary_op(*args)



------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legt model


#### LMU

In [14]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.0 transform
Creating HiPPO-lmu LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lmu model


#### LagT

In [15]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.0 transform
Creating HiPPO-lagt LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lagt model


#### FRU

In [16]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.0 transform
Creating HiPPO-fru LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-fru

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fru model


#### FouT

In [17]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.0 transform
Creating HiPPO-fout LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fout model


#### FouD

In [18]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.0 transform
Creating HiPPO-foud LTI model with 0.0 transform
Testing Coeffiecients for 0.0 LTI HiPPO-foud

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Backward Euler Transform

#### LegS

In [19]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------




Testing Coeffiecients for 1.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legs model


#### LegT

In [20]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 1.0 transform
Creating HiPPO-legt LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legt model


#### LMU

In [21]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 1.0 transform
Creating HiPPO-lmu LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lmu model


#### LagT

In [22]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 1.0 transform
Creating HiPPO-lagt LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lagt model


#### FRU

In [23]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 1.0 transform
Creating HiPPO-fru LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-fru

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fru model


#### FouT

In [24]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 1.0 transform
Creating HiPPO-fout LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fout model


#### FouD

In [25]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 1.0 transform
Creating HiPPO-foud LTI model with 1.0 transform
Testing Coeffiecients for 1.0 LTI HiPPO-foud

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Bidirectional Transform

#### LegS

In [26]:
test_operators(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------




Testing Coeffiecients for 0.5 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legs model


#### LegT

In [27]:
test_operators(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.5 transform
Creating HiPPO-legt LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legt model


#### LMU

In [28]:
test_operators(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.5 transform
Creating HiPPO-lmu LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lmu model


#### LagT

In [29]:
test_operators(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.5 transform
Creating HiPPO-lagt LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lagt model


#### FRU

In [30]:
test_operators(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.5 transform
Creating HiPPO-fru LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-fru

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fru model


#### FouT

In [31]:
test_operators(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.5 transform
Creating HiPPO-fout LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fout model


#### FouD

In [32]:
test_operators(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.5 transform
Creating HiPPO-foud LTI model with 0.5 transform
Testing Coeffiecients for 0.5 LTI HiPPO-foud

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With ZOH Transform

#### LegS

In [33]:
test_operators(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------




Testing Coeffiecients for 2.0 LSI HiPPO-legs

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legs model


#### LegT

In [34]:
test_operators(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legt LTI model with 2.0 transform
Creating HiPPO-legt LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-legt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-legt model


#### LMU

In [35]:
test_operators(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lmu LTI model with 2.0 transform
Creating HiPPO-lmu LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-lmu

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lmu model


#### LagT

In [36]:
test_operators(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lagt LTI model with 2.0 transform
Creating HiPPO-lagt LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-lagt

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-lagt model


#### FRU

In [37]:
test_operators(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fru LTI model with 2.0 transform
Creating HiPPO-fru LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-fru

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fru model


#### FouT

In [38]:
test_operators(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fout LTI model with 2.0 transform
Creating HiPPO-fout LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-fout

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-fout model


#### FouD

In [39]:
test_operators(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-foud LTI model with 2.0 transform
Creating HiPPO-foud LTI model with 2.0 transform
Testing Coeffiecients for 2.0 LTI HiPPO-foud

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------


end of test for HiPPO-foud model
